# Day 21
## Part 1

In [2]:
import numpy as np

In [3]:
with open("ex1.txt") as f:
# with open("starts.txt") as f:
    starts = [int(line.strip().split(": ")[1]) for line in f.readlines()]

print(starts)

[4, 8]


In [29]:
p1_space = starts[0]
p2_space = starts[1]
p1_score = 0
p2_score = 0
die = 1
roll_count = 0

# simulate game
while True:
    p1_space += (die*3 + 3)
    roll_count += 3
    if p1_space > 10:
        p1_space = ((p1_space - 1) % 10) + 1
    p1_score += p1_space
    if p1_score >= 1000:
        break
    die += 3
    
    p2_space += (die*3 + 3)
    roll_count += 3
    if p2_space > 10:
        p2_space = ((p2_space - 1) % 10) + 1
    p2_score += p2_space
    if p2_score >= 1000:
        break
    die += 3
    # break
    
print(p1_score, p2_score)
# print(roll_count)
print(roll_count * min(p1_score, p2_score))

1000 688
513936


## Part 2

In [35]:
# what if a player rolled 1 every time? how many turns would that take?
# maybe I should just deal with recursion now :/

def simulate_turn(player_space, player_score, roll, n_rolls):  # constant roll
    player_space += roll*3
    n_rolls += 3
    if player_space > 10:
        player_space = ((player_space - 1) % 10) + 1
    player_score += player_space
    return player_space, player_score, n_rolls
    
p1_score = 0
p1_space = starts[0]
p2_score = 0
p2_space = starts[1]
roll = 1
n_rolls = 0

while p1_score <= 21:
    p1_space, p1_score, n_rolls = simulate_turn(p1_space, p1_score, 1, n_rolls)
    p2_space, p2_score, n_rolls = simulate_turn(p2_space, p2_score, 1, n_rolls)
    
print(n_rolls)

24


Even in the worst case, there are only 24 rolls. Hmmm...

If there are only 24 rolls max, then I could just generate a list 24 long of numbers 1, 2, or 3. Then, simulate the game for each of these lists, making note of who won. Wait no, that wouldn't work :(

Maybe I should just do some sort of recursion thing that splits universes.

In [53]:
def split_universe(roll_history, n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score):
    # Pass in current rolls. If roll_history is a multiple of 3, then simulate a turn?
    # Since that means 3 rolls will have happened.
    if roll_history and len(roll_history) % 3 == 0:
        # A multiple of 3 rolls have been rolled
        if len(roll_history) % 6 == 0:
            # player 2 has just rolled - simulate turn using the last three rolls
            p2_space += sum(roll_history[-3:])
            if p2_space > 10:
                p2_space = ((p2_space - 1) % 10) + 1
            p2_score += p2_space
        else:
            # player 1 has just rolled - simulate turn using the last three rolls
            p1_space += sum(roll_history[-3:])
            if p1_space > 10:
                p1_space = ((p1_space - 1) % 10) + 1
            p1_score += p1_space
    
        if max(p1_score, p2_score) >= 21:
            # someone has win the game, so deal with that - return True or False maybe? 1 or 2? idk
            # return 1 if player 1 wins, then track the number of games played too.
            # can use difference to find games where player 2 wins.
            n_p1_wins += 1 if p1_score > p2_score else 0
            n_games += 1
            return n_p1_wins, n_games
        
    # no one has won, so roll another die
    # split the universe into 3
    n_p1_wins, n_games = split_universe(roll_history + [1], n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
    n_p1_wins, n_games = split_universe(roll_history + [2], n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
    n_p1_wins, n_games = split_universe(roll_history + [3], n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
        
    return n_p1_wins, n_games

In [65]:
roll_history = []
n_p1_wins = 0
n_games = 0
p1_space = starts[0]
p1_score = 0
p2_space = starts[1]
p2_score = 0

# n_p1_wins, n_games = split_universe(roll_history, n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
# print(n_p1_wins, n_games)
# print(n_games - n_p1_wins)

This is silly and will take forever.

Looking at how a turn works, all that really matters is the *sum* of the dice rolls, and not all sums are unique. If I only deal with the sums, that can cut down on the amount of work needed to be done.

In [1]:
import collections

unique_sums = collections.defaultdict(int)
for i in (1, 2, 3):
    for j in (1, 2, 3):
        for k in (1, 2, 3):
            unique_sums[i+j+k] += 1
            
for k, v in unique_sums.items():
    print(k, v)
    

print(sum(unique_sums.values()))

3 1
4 3
5 6
6 7
7 6
8 3
9 1
27


In [36]:
def split_universe(sum_history, n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score, unique_sums=unique_sums):
    # Pass in current sums of rolls. If odd number of sums, player 1s turn, else player 2s turn.
    history_length = len(sum_history)
    if history_length:
        if history_length % 2 == 0:
            # player 2 has just rolled - simulate turn using the last three rolls
            p2_space += sum_history[-1]
            if p2_space > 10:
                p2_space = ((p2_space - 1) % 10) + 1
            p2_score += p2_space
        elif history_length:
            # player 1 has just rolled - simulate turn using the last three rolls
            p1_space += sum_history[-1]
            if p1_space > 10:
                p1_space = ((p1_space - 1) % 10) + 1
            p1_score += p1_space

        if max(p1_score, p2_score) >= 21:
            # someone has won the game, so deal with that - return True or False maybe? 1 or 2? idk
            # return 1 if player 1 wins, then track the number of games played too.
            # can use difference to find games where player 2 wins.
            game_p1_wins = 1 if p1_score > p2_score else 0
            game_count = 1
            return n_p1_wins + (game_p1_wins * unique_sums[sum_history[-1]]), n_games + (game_count * unique_sums[sum_history[-1]])
        
    # no one has won, so add another sum.
    # there are only 7 unique sums.
    # maybe just write this one at a time tbh - "for loops are slow"
    for unique_sum, n_sum in unique_sums.items():
        # base_n_p1_wins, base_n_games = split_universe(sum_history + [unique_sum], n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
        # n_p1_wins += n_sum * base_n_p1_wins
        # n_games += n_sum * base_n_games
        n_p1_wins, n_games = split_universe(sum_history + [unique_sum], n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
        
    return n_p1_wins, n_games

In [51]:
sum_history = []
n_p1_wins = 0
n_games = 0
p1_space = starts[0]
p1_score = 21
p2_space = starts[1]
p2_score = 21

n_p1_wins, n_games = split_universe(sum_history, n_p1_wins, n_games, p1_space, p1_score, p2_space, p2_score)
print(n_p1_wins, n_games)
print(n_games - n_p1_wins)

# got the wrong answer :/
# don't know why :(
# I think my logic is sound, I just haven't written it right
# Maybe I should try doing this but just with n_games, until I get n_games right? hmm
# Maybe calculate from each space where it moves to next with each specific sum, just so I can get rid of the modulus stuff
# I don't know if that would speed it up but it might :O
# Maybe use numpy array with their positions too, idk

27 27
0


In [2]:
move_from = {}

for i in range(1, 11):
    # each space they move from
    move_to = {}
    for move, count in unique_sums.items():
        # how many spaces they will move
        move_to[move] = (((i + move) - 1) % 10) + 1
    move_from[i] = move_to

for k, v in move_from.items():
    print(k, v)

1 {3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10}
2 {3: 5, 4: 6, 5: 7, 6: 8, 7: 9, 8: 10, 9: 1}
3 {3: 6, 4: 7, 5: 8, 6: 9, 7: 10, 8: 1, 9: 2}
4 {3: 7, 4: 8, 5: 9, 6: 10, 7: 1, 8: 2, 9: 3}
5 {3: 8, 4: 9, 5: 10, 6: 1, 7: 2, 8: 3, 9: 4}
6 {3: 9, 4: 10, 5: 1, 6: 2, 7: 3, 8: 4, 9: 5}
7 {3: 10, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6}
8 {3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
9 {3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8}
10 {3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}


In [3]:
import functools

@functools.cache
def split_universe(roll_sum, isplayer1, p1_space, p1_score, p2_space, p2_score, n_p1_won=0, n_games=0):
    "Returns number of games ended and number of games in which player 1 won."
    if isplayer1:
        p1_space = move_from[p1_space][roll_sum]
        p1_score += p1_space
        if p1_score >= 21:
            # game end
            return 1, 1  # player1 won, game complete
    else:
        p2_space = move_from[p2_space][roll_sum]
        p2_score += p2_space
        if p2_score >= 21:
            return 0, 1
            # game end
    # no game end, so roll sum again
    for dice_sum, n_sum in unique_sums.items():
        result = split_universe(dice_sum, not isplayer1, p1_space, p1_score, p2_space, p2_score)
        n_p1_won += result[0] * n_sum
        n_games += result[1] * n_sum
    return n_p1_won, n_games

In [108]:
n_p1_won = 0
n_games = 0
isplayer1 = True
p1_space = starts[0]
p1_score = 0
p2_space = starts[1]
p2_score = 0

for dice_sum, n_sum in unique_sums.items():
    result = split_universe(dice_sum, isplayer1, p1_space, p1_score, p2_space, p2_score)
    n_p1_won += result[0] * n_sum
    n_games += result[1] * n_sum
    
print(f"P1 won {n_p1_won} times out of {n_games} games")
print(max(n_p1_won, n_games - n_p1_won))

P1 won 444356092776315 times out of 786316482957123 games
444356092776315


In [4]:
# with open("ex1.txt") as f:
with open("starts.txt") as f:
    starts = [int(line.strip().split(": ")[1]) for line in f.readlines()]

print(starts)

n_p1_won = 0
n_games = 0
isplayer1 = True
p1_space = starts[0]
p1_score = 0
p2_space = starts[1]
p2_score = 0

for dice_sum, n_sum in unique_sums.items():
    result = split_universe(dice_sum, isplayer1, p1_space, p1_score, p2_space, p2_score)
    n_p1_won += result[0] * n_sum
    n_games += result[1] * n_sum
    
print(f"P1 won {n_p1_won} times out of {n_games} games")
print(max(n_p1_won, n_games - n_p1_won))

[8, 2]
P1 won 105619718613031 times out of 199672040245315 games
105619718613031


In [13]:
# making my own cache function because I want to
SPLIT_UNIVERSE_CACHE = {}

def split_universe_my_cache(roll_sum, isplayer1, p1_space, p1_score, p2_space, p2_score, n_p1_won=0, n_games=0):
    "Returns number of games ended and number of games in which player 1 won."
    global SPLIT_UNIVERSE_CACHE
    check = hash(tuple([roll_sum, isplayer1, p1_space, p1_score, p2_space, p2_score]))
    
    if check in SPLIT_UNIVERSE_CACHE:
        return SPLIT_UNIVERSE_CACHE[check]
    
    if isplayer1:
        p1_space = move_from[p1_space][roll_sum]
        p1_score += p1_space
        if p1_score >= 21:
            # game end
            return 1, 1  # player1 won, game complete
    else:
        p2_space = move_from[p2_space][roll_sum]
        p2_score += p2_space
        if p2_score >= 21:
            return 0, 1
            # game end
    # no game end, so roll sum again
    for dice_sum, n_sum in unique_sums.items():
        result = split_universe_my_cache(dice_sum, not isplayer1, p1_space, p1_score, p2_space, p2_score)
        n_p1_won += result[0] * n_sum
        n_games += result[1] * n_sum
        
    SPLIT_UNIVERSE_CACHE[check] = [n_p1_won, n_games]

    return n_p1_won, n_games

In [15]:
with open("starts.txt") as f:
    starts = [int(line.strip().split(": ")[1]) for line in f.readlines()]

print(starts)

n_p1_won = 0
n_games = 0
isplayer1 = True
p1_space = starts[0]
p1_score = 0
p2_space = starts[1]
p2_score = 0

for dice_sum, n_sum in unique_sums.items():
    result = split_universe_my_cache(dice_sum, isplayer1, p1_space, p1_score, p2_space, p2_score)
    n_p1_won += result[0] * n_sum
    n_games += result[1] * n_sum
    
print(f"P1 won {n_p1_won} times out of {n_games} games")
print(max(n_p1_won, n_games - n_p1_won))

[8, 2]
P1 won 105619718613031 times out of 199672040245315 games
105619718613031
